## LangChain Expression Language (LCEL) for constituting chains

### chains are composed of **runnables** including models, prompts, parsers etc...

### parellelism, aSync, Batch processing

In [1]:
from dotenv import load_dotenv

google_api_key = load_dotenv("GOOGLE_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
model="gemini-1.5-flash"
parser= StrOutputParser()

llm= ChatGoogleGenerativeAI(model=model, gemini_api_key=google_api_key)


Unexpected argument 'gemini_api_key' provided to ChatGoogleGenerativeAI. Did you mean: 'google_api_key'?
c:\Users\ujwal\OneDrive\Documents\Python modules\PythonBasics_and_Modules\LangChain\venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: UserWarning: WARNING! gemini_api_key is not default parameter.
                gemini_api_key was transferred to model_kwargs.
                Please confirm that gemini_api_key is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
prompt= ChatPromptTemplate.from_template(
    "tell me s short joke about {topic}"
)
joke_chain= prompt | llm | parser

response= joke_chain.invoke({"topic": "cats"})
print("Joke:", response)

response2= joke_chain.invoke({"topic": "dogs"})
print("Joke:", response2)

Joke: Why are cats such bad dancers?  Because they have two left feet... and two more!
Joke: Why are dogs such bad dancers?  Because they have two left feet!


### pydantic parser

In [6]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from typing import List

class recipe(BaseModel):
    name: str = Field(description="Name of the recipe")
    ingredients: List[str] = Field(description="List of ingredients")
    instructions: str = Field(description="Instructions to prepare the recipe")
    prep_time_minutes: int = Field(description="Preparation time in minutes")

parser= PydanticOutputParser(pydantic_object=recipe)
format_isntruction= parser.get_format_instructions()
print(format_isntruction)

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that provides recipes."),
        ("human", "Give me a recipe for {dish} that takes less than {time} minutes to prepare.\n {format_instructions}"),
    ]
    ).partial(format_instructions=format_isntruction)

recipe_chain= prompt | llm | parser
response= recipe_chain.invoke({"dish": "pasta", "time": 30})
print("Recipe Name:", response.name)
print("Ingredients:", response.ingredients)
print("Instructions:", response.instructions)
print("Preparation Time (minutes):", response.prep_time_minutes)


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of the recipe", "title": "Name", "type": "string"}, "ingredients": {"description": "List of ingredients", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "instructions": {"description": "Instructions to prepare the recipe", "title": "Instructions", "type": "string"}, "prep_time_minutes": {"description": "Preparation time in minutes", "title": "Prep Time Minutes", "type": "integer"}}, "required": ["name", "ingredients", "instructions", "prep_time_minutes"]}
```
Recipe Name: Ga

### Streaming: parser runs only on full results

In [7]:
full_output=""
recipe_chain= prompt | llm 
for chunk in recipe_chain.stream({"dish": "pasta", "time": 30}):
    print(chunk, end="", flush=True)
    full_output += chunk.content
print("\nFull Output:", full_output)

parsed_response= parser.parse(full_output)
print("Parsed Recipe Name:", parsed_response.name)
print("Parsed Ingredients:", parsed_response.ingredients)
print("Parsed Instructions:", parsed_response.instructions)
print("Parsed Preparation Time (minutes):", parsed_response.prep_time_minutes)
print("Parsed Full Output:", parsed_response)

content='```' additional_kwargs={} response_metadata={'safety_ratings': []} id='run--ef27ffde-47ef-44d0-8758-254d89607142' usage_metadata={'input_tokens': 284, 'output_tokens': 0, 'total_tokens': 284, 'input_token_details': {'cache_read': 0}}content='json\n{\n  "name": "Quick Lemon Garlic Shrimp Pasta",\n' additional_kwargs={} response_metadata={'safety_ratings': []} id='run--ef27ffde-47ef-44d0-8758-254d89607142' usage_metadata={'output_tokens': 0, 'input_tokens': 0, 'total_tokens': 0, 'input_token_details': {'cache_read': 0}}content='  "ingredients": [\n    "1 pound linguine or spaghetti",\n' additional_kwargs={} response_metadata={'safety_ratings': []} id='run--ef27ffde-47ef-44d0-8758-254d89607142' usage_metadata={'output_tokens': 0, 'input_tokens': 0, 'total_tokens': 0, 'input_token_details': {'cache_read': 0}}content='    "1 tablespoon olive oil",\n    "4 cloves garlic, minced",\n    "1 pound shrimp, peeled and deveined",\n    "1' additional_kwargs={} response_metadata={'safety_rat

## Input/Output Schemas and RunnablePassthrough 

### Every Runnable in LCEL has defined input and output schemas

RunnablePassThrough:<br> Without RunnablePassthrough: You lose your original data when you transform it<br>
With RunnablePassthrough: You keep your original data AND get new computed data<br><br>Without RunnablePassthrough:
Input: "apple" → [make uppercase] → Output: "APPLE" (lost original)<br>
With RunnablePassthrough:
Input: {"fruit": "apple"} → [keep original + add uppercase] → Output: {"fruit": "apple", "uppercase": "APPLE"}

In [8]:
from langchain_core.runnables import RunnablePassthrough
summary_prompt= ChatPromptTemplate.from_template(
    "Summarize the following text:\n{text}\n\nSummary:"
)
question_prompt = ChatPromptTemplate.from_template(
    "Based on the original topic '{original_topic}', what is a related interesting fact?"
)
parser = StrOutputParser()

summary_chain = summary_prompt | llm | parser
# Use RunnablePassthrough to keep the original 'topic' and add the 'summary'
full_chain=(
    {"summary":summary_chain,"original_topic":RunnablePassthrough()}
    | question_prompt
    | llm
    | parser
)
input_data = {
    "text": "The Eiffel Tower is a wrought-iron lattice tower",
    "original_topic": "Eiffel Tower"
}
response = full_chain.invoke(input_data)
print("Summary:", response)



Summary: An interesting related fact is that **Gustave Eiffel's company wasn't originally intended to build the Eiffel Tower;  they won a design competition against over 100 other submissions.**  This highlights the competitive nature of its creation and the unexpected path to its construction.


.assign()

In [6]:
from langchain_core.runnables import RunnablePassthrough
passthroug= RunnablePassthrough()

#result=passthroug.invoke("hello world")
result="hello world"
print("Passthrough Result:", result)

chain=RunnablePassthrough.assign(
    upper_case=lambda x: x["text"].upper(),
    length=lambda x: len(x["text"]),
    )
result=chain.invoke({"text": "hello world"})
print(result)

Passthrough Result: hello world
{'text': 'hello world', 'upper_case': 'HELLO WORLD', 'length': 11}


## Branching <br> 

### Branching allows you to split execution into multiple parallel paths, each processing the same input differently.

In [9]:
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate
summary_prompt= PromptTemplate.from_template(
    "Summarize the following text:\n{text}\n\nSummary:"
)
keywords_prompt= PromptTemplate.from_template(
    "Extract keywords from the following text:\n{text}\n\nKeywords:"    
)
sentiment_prompt= PromptTemplate.from_template(
    "Analyze the sentiment of the following text:\n{text}\n\nSentiment:"
)

branch_chain= RunnableParallel({
    "summary": summary_prompt | llm,
    "keywords": keywords_prompt | llm,
    "sentiment": sentiment_prompt | llm,
    "original_text": RunnablePassthrough()
})

result=branch_chain.invoke({
    "text": "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.",
    "original_text": "Eiffel Tower"
})

print(result)

{'summary': AIMessage(content='The Eiffel Tower is a wrought-iron structure located in Paris, France.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--26b5c2e5-c6e0-43ac-a1e4-b6418d77b295-0', usage_metadata={'input_tokens': 30, 'output_tokens': 16, 'total_tokens': 46, 'input_token_details': {'cache_read': 0}}), 'keywords': AIMessage(content='Keywords: Eiffel Tower, wrought-iron, lattice tower, Champ de Mars, Paris, France', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--44b4a6ed-f5a1-426c-ab38-e2999d893fb4-0', usage_metadata={'input_tokens': 31, 'output_tokens': 20, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}}), 'sentiment': AIMessage(content='Sentiment: Neutral.\n\nThe text provi

output is stored in dict with the name of runnable path as key and output as a value

In [16]:
joke_prompt= ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

fact_prompt= ChatPromptTemplate.from_template(
    "tell me an interesting fact about {topic}"
)

parellel_chain=RunnableParallel(
    joke=joke_prompt | llm | parser,
    fact=fact_prompt | llm | parser,
)

final_chain=(
    {"topic":RunnablePassthrough()}
    | parellel_chain
    | (lambda x: f"Joke: {x['joke']}\nFact: {x['fact']}")
)

topic = "cats"
response = final_chain.invoke({"topic": topic})
print("Final Response:", response)

Final Response: Joke: Why are cats such bad dancers?  Because they have two left feet!
Fact: A cat's purr is not just a cute sound;  the frequency of a cat's purr (25-150 Hz) has been shown to promote bone and tissue healing in humans and cats themselves.  This is why you might find a purring cat recovering from injury more quickly.


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch

# Define prompts
cat_prompt = ChatPromptTemplate.from_template(
    "Tell me a joke about {animal}."
)
dog_prompt = ChatPromptTemplate.from_template(
    "Tell me a joke about {animal}."
)

# Create a default prompt for other animals
default_prompt = ChatPromptTemplate.from_template(
    "I don't have a specific joke for {animal}, but here's a general one: Why did the chicken cross the road? To get to the other side!"
)

# Create chains
cat_chain = cat_prompt | llm
dog_chain = dog_prompt | llm
default_chain = default_prompt | llm

# Create RunnableBranch with default
branch_chain = RunnableBranch(
    (lambda x: "cat" in x["animal"].lower(), cat_chain),
    (lambda x: "dog" in x["animal"].lower(), dog_chain),
    default_chain  # This is the required default branch
)

# Test the branch
response = branch_chain.invoke({"animal": "cat"})
print("Branch Response (Cat):", response)

response = branch_chain.invoke({"animal": "dog"})
print("Branch Response (Dog):", response)

response = branch_chain.invoke({"animal": "elephant"})
print("Branch Response (Default):", response)

Branch Response (Cat): content='Why was the cat sitting on the computer?  To keep an eye on the mouse!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run--b3427cb2-4dea-4baf-aff4-aa9d9dfd5657-0' usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}


KeyboardInterrupt: 